# 3/ Automated product review and classification with SQL functions


In this demo, we will explore the SQL AI function `ai_query` to create a pipeline extracting product review information.

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/product/sql-ai-functions/sql-ai-query-function-flow.png" width="1000">

**Don't forget about built-in SQL AI functions!** *In this notebook, we show you how to create your own custom functions. However, many text-related tasks (translation, classification etc.) are available as [builtin SQL functions]($./01-Builtin-SQL-AI-Functions). If you can, prefere these as they're easy to use and performant!*

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=DBSQL&org_id=1444828305810485&notebook=%2F03-automated-product-review-and-answer&demo_name=sql-ai-functions&event=VIEW&path=%2F_dbdemos%2FDBSQL%2Fsql-ai-functions%2F03-automated-product-review-and-answer&version=1">

To run this notebook, connect to <b> SQL endpoint </b>. The AI_QUERY function is available on Databricks SQL Pro and Serverless.

In [0]:
-- as previously, make sure you run this notebook using a SQL Warehouse or Serverless endpoint (not a classic cluster)
SELECT assert_true(current_version().dbsql_version is not null, 'YOU MUST USE A SQL WAREHOUSE OR SERVERLESS, not a classic cluster');

USE CATALOG mozuca;
USE SCHEMA did2025;


## Simplifying AI function access for SQL users

As reminder, `ai_query` signature is the following:

```
SELECT ai_query(<Endpoint Name>, <prompt>)
```

In the [previous notebook]($./02-Generate-fake-data-with-AI-functions-Foundation-Model), we created a wrapper `ASK_LLM_MODEL` function to simplify our SQL operation and hide the configuration details to end-users. We will re-use this function for this pipeline.

In order to simplify the user-experience for our analysts, we will build prescriptive SQL functions that ask natural language questions of our data and return the responses as structured data.

In [0]:
SELECT * FROM fake_reviews INNER JOIN fake_customers using (customer_id)

## Review analysis with prompt engineering 
&nbsp;
The keys to getting useful results back from a LLM model are:
- Asking it a well-formed question
- Being specific about the type of answer that you are expecting

In order to get results in a form that we can easily store in a table, we'll ask the model to return the result in a string that reflects `JSON` representation, and be very specific of the schema that we expect

Here's the prompt we've settled on:
```
A customer left a review on a product. We want to follow up with anyone who appears unhappy.
Extract all entities mentioned. For each entity:
- classify sentiment as ["POSITIVE","NEUTRAL","NEGATIVE"]
- whether customer requires a follow-up: Y or N
- reason for requiring followup

Return JSON ONLY. No other text outside the JSON. JSON format:
[{
    "product_name": <product name>,
    "category": <product category>,
    "sentiment": <review sentiment, one of ["POSITIVE","NEUTRAL","NEGATIVE"]>,
    "followup": <Y or N for follow up>,
    "followup_reason": <reason for followup>
}]

Review:
<insert review text here>
```

In [0]:
CREATE OR REPLACE FUNCTION ANNOTATE_REVIEW(review STRING)
    RETURNS STRUCT<product_name: STRING, entity_sentiment: STRING, followup: STRING, followup_reason: STRING>
    RETURN FROM_JSON(
      ASK_LLM_MODEL(CONCAT(
        'A customer left a review. We follow up with anyone who appears unhappy.
         extract the following information:
          - classify sentiment as ["POSITIVE","NEUTRAL","NEGATIVE"]
          - returns whether customer requires a follow-up: Y or N
          - if followup is required, explain what is the main reason

        Return JSON ONLY. No other text outside the JSON. JSON format:
        {
            "product_name": <entity name>,
            "entity_sentiment": <entity sentiment>,
            "followup": <Y or N for follow up>,
            "followup_reason": <reason for followup>
        }
        
        Review:', review), "{'type': 'json_object'}"),
      "STRUCT<product_name: STRING, entity_sentiment: STRING, followup: STRING, followup_reason: STRING>")

-- ALTER FUNCTION ANNOTATE_REVIEW OWNER TO `your_principal`; -- for the demo only, make sure other users can access your function

In [0]:
CREATE OR REPLACE TABLE reviews_annotated as 
    SELECT * EXCEPT (review_annotated), review_annotated.* FROM (
      SELECT *, ANNOTATE_REVIEW(review) AS review_annotated
        FROM fake_reviews)
    INNER JOIN fake_customers using (customer_id)

In [0]:
SELECT * FROM reviews_annotated

In [0]:
CREATE OR REPLACE FUNCTION GENERATE_RESPONSE(firstname STRING, lastname STRING, order_count INT, product_name STRING, reason STRING)
  RETURNS STRING
  RETURN ASK_LLM_MODEL(
    CONCAT("Our customer named ", firstname, " ", lastname, " who ordered ", order_count, " ", product_name, " was unhappy about ", product_name, "specifically due to ", reason, ". Provide an empathetic message I can send to my customer 
    including the offer to have a call with the relevant product manager to leave feedback. I want to win back their 
    favour and I do not want the customer to churn"), 
    "{'type': 'text'}"
  );
-- ALTER FUNCTION GENERATE_RESPONSE OWNER TO `account users`; -- for the demo only, make sure other users can access your function

In [0]:
SELECT GENERATE_RESPONSE("Quentin", "Ambard", 235, "Country Choice Snacking Cookies", "Quality issue") AS customer_response

In [0]:
CREATE OR REPLACE TABLE reviews_answer as 
    SELECT *,
      generate_response(firstname, lastname, order_count, product_name, followup_reason) AS response_draft
    FROM reviews_annotated where followup='Y'

In [0]:
SELECT * FROM reviews_answer

### Creating our Customer Review Dashboard

The next step is to set up a comprehensive dashboard in order to track and monitor our customer reviews.

Open the <a dbdemos-dashboard-id="customer-review-analysis" href='/sql/dashboardsv3/01f0095cfcf3192da39df151d743c498' target="_blank">Customer review analysis dashboard</a> to have a complete view of your customers, products and reviews.

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/product/sql-ai-functions/sql-ai-function-dashboard.png" width="1200">


### Going further

Our pipeline is ready. Keep in mind that this is a fairly basic pipeline for our demo.

For more advanced pipeline, we recommend using Delta Live Table. DLT simplify data ingetsion and transformation tasks with incremental load, materialized view and more advanced features. For more details, run `dbdemos.install_demo('dlt-loans')`

### Extra: AdHoc Queries

Remember that analysts can always use the `ASK_LLM_MODEL()` function we created earlier to apply their own prompts to the data.

As short example, let's write a query to extract all review about beverages:

In [0]:
SELECT review_id,
    BOOLEAN(ASK_LLM_MODEL(
      CONCAT("Does this review discuss beverages? Answer boolean: 'true' or 'false' only, lowercase, no explanations or notes nor final dot. Review: ", review)
    )) AS is_beverage_review,
    review
  FROM fake_reviews LIMIT 10

### Extra: Create an Production Ready Pipeline using Delta Live Table

We can turn the steps in this notebook into a production ready Delta Live Table pipeline with AI SQL Functions

Open [04-create-end-to-end-DLT-workflow]($./04-create-end-to-end-DLT-workflow) for more details.

## You're now ready to process your text using external LLM models!

We've seen that the lakehouse provide advanced AI capabilities, not only you can leverage external LLM APIs, but you can also build your own LLM with Databricks GenAI applications!
For more details on creating your chatbot with the Lakehouse, run: `dbdemos.install('llm-rag-chatbot')`

Go back to [the introduction]($./01-SQL-AI-Functions-Introduction)

## Extra: configure an External Model Endpoint to leverage external providers (OpenAI, Anthropic...) 

This demo was using one Databricks Foundation Model (pricing token-based).

Your model endpoint can also be setup to use an external model such as OpenAI. Open [05-Extra-setup-external-model-OpenAI]($./05-Extra-setup-external-model-OpenAI) for more details.


Go back to [the introduction]($./00-SQL-AI-Functions-Introduction)